In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
from langchain_teddynote import logging

logging.langsmith("CH01-Basic",set_enable=True)

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


In [4]:
from langchain_core.prompts import PromptTemplate

### 1. llm에 질문하는 프롬프트 만들기
1. 변수가 들어가는 문자열 변수 생성
2. PromptTemplate.from_template으로 PromptTemplate 객체생성
2-1. PromptTemplate 객체는 변수에 값을 채워 동적으로 프롬프트를 작성 가능함.

In [8]:
template = "안녕하세요는 {country}로 어떻게 말해?"

In [9]:
prompt = PromptTemplate.from_template(template)

In [10]:
english_prompt = prompt.format(country = "영어")
japanes_prompt = prompt.format(country = "일본어")

print(english_prompt)
print(japanes_prompt)

안녕하세요는 영어로 어떻게 말해?
안녕하세요는 일본어로 어떻게 말해?


### 2. 모델 생성 및 호출
1. ChatOpenAI는 openAI로 모델을 선언함. 다양한 모델은 향후 학습
2. .invoke(input) 모델을 호출하는 명령어
2-1. input에 프롬프트를 삽입
2-2. 원하는 모델에 .invoke()사용으로 질문에 대한 답변을 호출

In [23]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name = "gpt-4o-mini")


In [26]:
model.invoke(japanes_prompt)

AIMessage(content='안녕하세요는 일본어로 "こんにちは" (곤니치와)라고 말합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 17, 'total_tokens': 36, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b78b61c52', 'finish_reason': 'stop', 'logprobs': None}, id='run-e653ed05-3604-4903-bb41-ebb3e7c1f1ec-0', usage_metadata={'input_tokens': 17, 'output_tokens': 19, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

### 3. Chain



In [27]:
chain = prompt|model

In [29]:
chain.invoke({"country": "스페인어"})

AIMessage(content='안녕하세요는 스페인어로 "Hola"라고 말합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 19, 'total_tokens': 34, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bfc6a7dc2', 'finish_reason': 'stop', 'logprobs': None}, id='run-bba3d208-28bb-4417-9a3b-286bdfc0693b-0', usage_metadata={'input_tokens': 19, 'output_tokens': 15, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [37]:
from langchain_core.messages import AIMessageChunk
response = chain.stream({"country": "스페인어"})


def stream(response):
    answer = ""
    for token in response:
        if isinstance(token, AIMessageChunk):
                answer += token.content
                print(token.content, end="", flush=True)
        elif isinstance(token, str):
                answer += token
                print(token, end="", flush=True)

### 출력 파서 StrOutputParser 결과를 문자열 형식으로 출력

In [33]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()


In [35]:
template = """
    너는 question에 상황을 학습용 영어회화를 해주는 어시스턴트야.
    question에 [FORMAT]에 영어회화를 작성해.

    question : {question}

    FORMAT:
    - 영어 회화:
    - 한글 해석:

"""

prompt = PromptTemplate.from_template(template)


In [38]:
chain = prompt|model|output_parser
response = chain.invoke({"question":"저는 안내데스크에서 예약한 비행기표를 받고 싶어요."})

stream(response)

- 영어 회화: 
  A: Hi, I’d like to pick up my flight ticket that I reserved.  
  B: Sure! Can I have your name and reservation number, please?  
  A: My name is [Your Name], and my reservation number is [Your Reservation Number].  
  B: Thank you! Here’s your ticket. Have a great flight!  

- 한글 해석:  
  A: 안녕하세요, 제가 예약한 비행기표를 받고 싶어요.  
  B: 물론이죠! 이름과 예약 번호를 말씀해 주실 수 있나요?  
  A: 제 이름은 [당신의 이름]이고, 예약 번호는 [당신의 예약 번호]입니다.  
  B: 감사합니다! 여기 당신의 비행기표입니다. 좋은 비행 되세요!